In [ ]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 8.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import random
import warnings
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, precision_score, accuracy_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neural_network import MLPClassifier
from deap import base, creator, tools, algorithms

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_parquet('enade_cenario_1_3_faixas.parquet.gzip')
df.head()

,REGIAO_IES,SEXO,COR_RACA,ESTADO_CIVIL,RENDA_FAMILIAR,ESCOLARIDADE_PAI,ESCOLARIDADE_MAE,COMPANHIA_RESIDENCIA,TIPO_ESCOLA_ENSINO_MEDIO,MODALIDADE_ENSINO_MEDIO,...,OFERTA_APOIO_GERAL,SITUACAO_TRABALHO,SITUACAO_FINANCEIRA,DEDICACAO_SEMANAL,GRAU_ACADEMICO_CURSO,REGIAO_ESCOLA_ENSINO_MEDIO,GRANDE_AREA_CURSO,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO,TEMPO_GRADUACAO_1,FAIXA_ETARIA
0,REGIAO_IES: Sul,SEXO: Masculino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),"RENDA_FAMILIAR: AtÃ© 1,5 salÃ¡rio mÃ­nimo (atÃ...",ESCOLARIDADE_PAI: Nenhuma,ESCOLARIDADE_MAE: Nenhuma,"COMPANHIA_RESIDENCIA: Em casa ou apartamento, ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino mÃ©dio tradici...,...,OFERTA_APOIO_GERAL: NÃ£o,SITUACAO_TRABALHO: NÃ£o,SITUACAO_FINANCEIRA: NÃ£o tenho renda e meus g...,"DEDICACAO_SEMANAL: Nenhuma, apenas assisto Ã s...",GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Norte,GRANDE_AREA_CURSO: ServiÃ§os,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: 2...,TEMPO_GRADUACAO: Entre 2 e 4 anos,FAIXA_ETARIA: Entre 18 e 24 anos
1,REGIAO_IES: Sul,SEXO: Feminino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),"RENDA_FAMILIAR: De 3 a 4,5 salÃ¡rios mÃ­nimos ...",ESCOLARIDADE_PAI: Ensino mÃ©dio,ESCOLARIDADE_MAE: Ensino superior,"COMPANHIA_RESIDENCIA: Em casa ou apartamento, ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino mÃ©dio tradici...,...,OFERTA_APOIO_GERAL: NÃ£o,SITUACAO_TRABALHO: Sim,SITUACAO_FINANCEIRA: Tenho renda e contribuo p...,DEDICACAO_SEMANAL: De uma a trÃªs,GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Sul,GRANDE_AREA_CURSO: ServiÃ§os,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: 2...,TEMPO_GRADUACAO: Entre 5 e 7 anos,FAIXA_ETARIA: Entre 25 e 39 anos
2,REGIAO_IES: Sul,SEXO: Feminino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),"RENDA_FAMILIAR: AtÃ© 1,5 salÃ¡rio mÃ­nimo (atÃ...",ESCOLARIDADE_PAI: Ensino mÃ©dio,ESCOLARIDADE_MAE: Ensino mÃ©dio,"COMPANHIA_RESIDENCIA: Em casa ou apartamento, ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino mÃ©dio tradici...,...,OFERTA_APOIO_GERAL: Sim,SITUACAO_TRABALHO: NÃ£o,"SITUACAO_FINANCEIRA: Tenho renda, mas recebo a...",DEDICACAO_SEMANAL: De quatro a sete,GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Sul,GRANDE_AREA_CURSO: ServiÃ§os,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: E...,TEMPO_GRADUACAO: Entre 2 e 4 anos,FAIXA_ETARIA: Entre 18 e 24 anos
3,REGIAO_IES: Sul,SEXO: Feminino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),"RENDA_FAMILIAR: AtÃ© 1,5 salÃ¡rio mÃ­nimo (atÃ...",ESCOLARIDADE_PAI: Nenhuma,ESCOLARIDADE_MAE: Nenhuma,"COMPANHIA_RESIDENCIA: Em casa ou apartamento, ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino mÃ©dio tradici...,...,OFERTA_APOIO_GERAL: NÃ£o,SITUACAO_TRABALHO: NÃ£o,SITUACAO_FINANCEIRA: NÃ£o tenho renda e meus g...,"DEDICACAO_SEMANAL: Nenhuma, apenas assisto Ã s...",GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Norte,GRANDE_AREA_CURSO: ServiÃ§os,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: E...,TEMPO_GRADUACAO: Entre 5 e 7 anos,FAIXA_ETARIA: Entre 25 e 39 anos
4,REGIAO_IES: Sul,SEXO: Feminino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),RENDA_FAMILIAR: De 10 a 30 salÃ¡rios mÃ­nimos ...,ESCOLARIDADE_PAI: Ensino superior,ESCOLARIDADE_MAE: Ensino superior,"COMPANHIA_RESIDENCIA: Em casa, apartamento ou ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino mÃ©dio tradici...,...,OFERTA_APOIO_GERAL: Sim,SITUACAO_TRABALHO: Sim,"SITUACAO_FINANCEIRA: Tenho renda, mas recebo a...",DEDICACAO_SEMANAL: De uma a trÃªs,GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Sudeste,GRANDE_AREA_CURSO: ServiÃ§os,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: 2...,TEMPO_GRADUACAO: Entre 2 e 4 anos,FAIXA_ETARIA: Entre 18 e 24 anos


In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse = False)
X = enc.fit_transform(df.drop('TEMPO_GRADUACAO_1', axis = 1))
y = df['TEMPO_GRADUACAO_1']

In [ ]:
rus = RandomUnderSampler(random_state=0)
X, y = rus.fit_resample(X, y)

In [ ]:
X

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [ ]:
y

0          TEMPO_GRADUACAO: 8 anos ou mais
1          TEMPO_GRADUACAO: 8 anos ou mais
2          TEMPO_GRADUACAO: 8 anos ou mais
3          TEMPO_GRADUACAO: 8 anos ou mais
4          TEMPO_GRADUACAO: 8 anos ou mais
                       ...                
27565    TEMPO_GRADUACAO: Entre 5 e 7 anos
27566    TEMPO_GRADUACAO: Entre 5 e 7 anos
27567    TEMPO_GRADUACAO: Entre 5 e 7 anos
27568    TEMPO_GRADUACAO: Entre 5 e 7 anos
27569    TEMPO_GRADUACAO: Entre 5 e 7 anos
Name: TEMPO_GRADUACAO_1, Length: 27570, dtype: object

In [ ]:
one_hot_encoded_frame =  pd.DataFrame(X, columns= enc.get_feature_names())

In [ ]:
model = MLPClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(one_hot_encoded_frame, y, test_size=0.3)

In [ ]:
clf_full = model.fit(X_train,y_train)
y_pred = clf_full.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.5559182686494982


In [ ]:
# Create a fitness function with weight 1. 
# If it's a maximization function the weight should be positive else,for minimization, it should be negative 
# Here, we want to maximize accuracy
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# Create blank individual as list with the fitness function defined above.
# Individual is synonymous with chromosome
creator.create("Individual", list, fitness=creator.FitnessMax)
# Declare the size of the chromosome. Here the size will be no. of variables
ind_size = X_train.shape[1]
# Declare the toolbox. Toolbox is a container which contains all the operators declared
# To include any operator in the toolbox, it has to registered
toolbox = base.Toolbox()
# Register a random number generator function to randomly generate 1 or 0.
# This will signify whether a variable is present or not in a particular individual. 1: Present; 0: Absent
# 1/0 values, here, are alleles
toolbox.register("attrib_bin", random.randint, 0, 1)
# Insert values in individuals which will have value from random number generator defined above
# size of each individual will be no. of variables
# Register the individuals in the toolbox
# initRepeat argument is used to initialize the individual
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attrib_bin, n=ind_size)
# Define a population with the individuals created above
# Register the population in toolbox
# initRepeat argument is used to initialize the population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
# ind1 = toolbox.individual()
print("Total no. of variables/length of each chromosome = ", ind_size)

Total no. of variables/length of each chromosome =  116


In [ ]:
def evaluate(individual):
    sum_features = np.sum(individual)
    if sum_features == 0:
        return 0.0
    else:
        sum_features = np.sum(individual)
        collector_train = []
        collector_test = []
        for k in range(0, len(individual)):
            if individual[k] == 1:
                collector_train.append(X_train.iloc[:, k])
                collector_test.append(X_test.iloc[:, k])
        X_train1 = np.transpose(np.array(collector_train))
        X_test1 = np.transpose(np.array(collector_test))
        np.nan_to_num(X_train1, copy=False)
        np.nan_to_num(y_train, copy=False)
        np.nan_to_num(X_test1, copy=False)
        np.nan_to_num(y_test, copy=False)
        clf = model.fit(X_train1, y_train)
        y_pred = clf.predict(X_test1)
        # accuracy = (y_test['target'].tolist()==y_pred).sum() / y_test['target'].count()
        accuracy = accuracy_score(y_test, y_pred)
        return (accuracy,)

In [ ]:
# Two points crossover
toolbox.register("mate", tools.cxTwoPoint)
# Bit flip mutation The indpb argument is the probability of each attribute to be flipped
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
# Select the best individual among tournsize randomly chosen individuals
toolbox.register("select", tools.selTournament, tournsize=3)
# Register the fitness function defined above in the toolbox
toolbox.register("evaluate", evaluate)
# Define the statistics to be shown during the algorithm run.
# We have selected minimum, maximum and average accuracy for each generation of run
# Decision will, however, be taken based on maximum accuracy as defined earlier
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Min", np.min)

In [ ]:
#Define initial population with its size. Here we are defining low population size (only 20) for demonstration
pop = toolbox.population(n=20)
# Below HallOfFame argument saves k best individual across all the generations
# We will use this to determine our final variable combination. 
# We are taking only the first best one. It is possible to save k number of best solution in hof
hof = tools.HallOfFame(1)
# Now, let's run the algorithm with the most simple in-built method available in DEAP (eaSimple).
# The arguments are as below:
# pop: Population defined earlier
# toolbox: toolbox containing all the operator defined
# cxpb: The probability of mating two individuals.
# mutpb: The probability of mutating an individual. We are keeping it high to show the impact
# ngen: The number of generation.
pop, log = algorithms.eaSimple(
    pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=30, halloffame=hof, stats=stats, verbose=True)
# Get the best individual
best = hof.items[0]
#Print the best individual and it's accuracy
print("Best Individual = ", best)
print("Best Fitness = ", best.fitness.values[0])

gen	nevals	Mean    	Max     	Min     
0  	20    	0.545442	0.588683	0.467779
1  	16    	0.565361	0.585782	0.491597
2  	13    	0.568178	0.579011	0.551808
3  	15    	0.571219	0.58687 	0.505985
4  	15    	0.576986	0.587112	0.559787
5  	16    	0.580583	0.59219 	0.569701
6  	16    	0.582342	0.597993	0.56148 
7  	7     	0.588381	0.597993	0.571636
8  	16    	0.5847  	0.595454	0.57224 
9  	13    	0.586023	0.596059	0.553742
10 	12    	0.586997	0.596059	0.574296
11 	13    	0.583454	0.596905	0.504292
12 	11    	0.5871  	0.594729	0.571757
13 	16    	0.586948	0.595696	0.567767
14 	12    	0.589227	0.599202	0.576835
15 	13    	0.588986	0.597993	0.575626
16 	15    	0.590576	0.601862	0.569097
17 	10    	0.592728	0.601862	0.574054
18 	13    	0.591373	0.601862	0.581308
19 	14    	0.591035	0.601862	0.580704
20 	18    	0.587093	0.59896 	0.575626
21 	17    	0.589542	0.599202	0.580462
22 	8     	0.592329	0.604643	0.576351
23 	14    	0.592746	0.604643	0.582155
24 	14    	0.594136	0.604643	0.57889 
25 	14    	0

In [ ]:
counter_1 = 0
for i in best:
  if i == 1:
    counter_1 += 1
print(counter_1)

65


In [ ]:
best_collector = []
for k in range(0, 116):
  if best[k] == 1:
    best_collector.append(X_train.columns[k])
print("No. of variables select = ", len(best_collector))
print(best_collector)

No. of variables select =  65
['x0_REGIAO_IES: Centro-Oeste', 'x0_REGIAO_IES: Sudeste', 'x0_REGIAO_IES: Sul', 'x1_SEXO: Feminino', 'x2_COR_RACA: IndÃ\xadgena', 'x3_ESTADO_CIVIL: Outro', 'x3_ESTADO_CIVIL: Separado(a) judicialmente/divorciado(a)', 'x3_ESTADO_CIVIL: Solteiro(a)', 'x4_RENDA_FAMILIAR: De 1,5 a 3 salÃ¡rios mÃ\xadnimos (R$ 1.431,01 a R$ 2.862,00)', 'x4_RENDA_FAMILIAR: De 3 a 4,5 salÃ¡rios mÃ\xadnimos (R$ 2.862,01 a R$ 4.293,00)', 'x5_ESCOLARIDADE_PAI: Nenhuma', 'x6_ESCOLARIDADE_MAE: Ensino mÃ©dio', 'x6_ESCOLARIDADE_MAE: Ensino superior', 'x7_COMPANHIA_RESIDENCIA: Em alojamento universitÃ¡rio da prÃ³pria instituiÃ§Ã£o', 'x7_COMPANHIA_RESIDENCIA: Em casa ou apartamento, com cÃ´njuge e/ou filhos', 'x7_COMPANHIA_RESIDENCIA: Em casa ou apartamento, sozinho', 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior parte em escola privada', 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior parte em escola pÃºblica', 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou parte no exterior', 'x9_MODALIDADE_ENSINO_M

In [ ]:
best_collector

['x0_REGIAO_IES: Centro-Oeste',
 'x0_REGIAO_IES: Sudeste',
 'x0_REGIAO_IES: Sul',
 'x1_SEXO: Feminino',
 'x2_COR_RACA: IndÃ\xadgena',
 'x3_ESTADO_CIVIL: Outro',
 'x3_ESTADO_CIVIL: Separado(a) judicialmente/divorciado(a)',
 'x3_ESTADO_CIVIL: Solteiro(a)',
 'x4_RENDA_FAMILIAR: De 1,5 a 3 salÃ¡rios mÃ\xadnimos (R$ 1.431,01 a R$ 2.862,00)',
 'x4_RENDA_FAMILIAR: De 3 a 4,5 salÃ¡rios mÃ\xadnimos (R$ 2.862,01 a R$ 4.293,00)',
 'x5_ESCOLARIDADE_PAI: Nenhuma',
 'x6_ESCOLARIDADE_MAE: Ensino mÃ©dio',
 'x6_ESCOLARIDADE_MAE: Ensino superior',
 'x7_COMPANHIA_RESIDENCIA: Em alojamento universitÃ¡rio da prÃ³pria instituiÃ§Ã£o',
 'x7_COMPANHIA_RESIDENCIA: Em casa ou apartamento, com cÃ´njuge e/ou filhos',
 'x7_COMPANHIA_RESIDENCIA: Em casa ou apartamento, sozinho',
 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior parte em escola privada',
 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior parte em escola pÃºblica',
 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou parte no exterior',
 'x9_MODALIDADE_ENSINO_MEDIO: Educa

In [ ]:
X_train1_best = X_train[best_collector]
clf_best = model.fit(X_train1_best,y_train)
X_test1 = X_test[best_collector]
y_pred = clf_best.predict(X_test1)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.5718776447829766


In [ ]:
0.5937613347841857

In [ ]:
clf_full = model.fit(X_train,y_train)
y_pred = clf_full.predict(X_test)
print(classification_report(y_test, y_pred))

                                   precision    recall  f1-score   support

  TEMPO_GRADUACAO: 8 anos ou mais       0.62      0.68      0.65      2715
TEMPO_GRADUACAO: Entre 2 e 4 anos       0.62      0.62      0.62      2810
TEMPO_GRADUACAO: Entre 5 e 7 anos       0.47      0.42      0.44      2746

                         accuracy                           0.57      8271
                        macro avg       0.57      0.57      0.57      8271
                     weighted avg       0.57      0.57      0.57      8271

